In [ ]:
img_size = 128
class_names = ["Angry", "Fear", "Happy", "Sad", "Suprise"]  # sorted folder names

In [ ]:
# app_gradio.py
from typing import List
import sys

import tensorflow as tf
from tensorflow import keras

# --------------------------- Gradio (with consent, blur-preview) ---------------------------
def launch_gradio(model: keras.Model, class_names: List[str], img_size: int) -> None:
    try:
        import gradio as gr
    except Exception:
        sys.stderr.write("Gradio not installed. Try: pip install gradio\n")
        return

    # Optional CV dependency (faster blur/resize). Falls back to PIL if missing.
    try:
        import cv2  # type: ignore
    except Exception:
        cv2 = None

    import numpy as np
    from PIL import Image, ImageFilter

    PRIVACY_TEXT = """
### Privacy & Consent
By proceeding, you confirm you have the right to upload this image and consent to its processing **in this session** for emotion classification.
- Images are processed in-session; no retention beyond this session.
- Do not upload sensitive/identifiable images without permission.
"""

    def _resize_np(img_np, size: int):
        if cv2 is not None:
            return cv2.resize(img_np, (size, size), interpolation=cv2.INTER_AREA)
        return np.array(Image.fromarray(img_np).resize((size, size)))

    def _maybe_blur_preview(pil_img: Image.Image, blur: bool) -> Image.Image:
        # UI preview anonymization only; model runs on original unless you change logic below
        return pil_img.filter(ImageFilter.GaussianBlur(radius=12)) if blur else pil_img

    def _predict_core(pil_img: Image.Image):
        img_np = np.array(pil_img.convert("RGB"))
        img_np = _resize_np(img_np, img_size)
        x = (img_np.astype(np.float32) / 255.0)[None, ...]
        probs = model.predict(x, verbose=0)[0]
        pred_id = int(np.argmax(probs))
        pred_label = class_names[pred_id]
        probs_dict = {class_names[i]: float(probs[i]) for i in range(len(class_names))}
        return probs_dict, pred_label

    def toggle_uploader(consent: bool):
        if not consent:
            # Hide and reset everything when consent revoked
            return (
                gr.update(visible=False, value=None),  # file
                gr.update(value=None),                 # preview
                gr.update(value={}),                   # probs
                gr.update(value=""),                   # label
            )
        return (gr.update(visible=True), gr.update(), gr.update(), gr.update())

    def process_file(file_obj, consent: bool, anonymize: bool):
        if not consent:
            return None, {}, "Consent required"
        if file_obj is None:
            return None, {}, "No image"
        try:
            pil = Image.open(file_obj.name).convert("RGB")
        except Exception:
            return None, {}, "Invalid image"
        preview = _maybe_blur_preview(pil, anonymize)
        probs, pred = _predict_core(pil)  # model on original (not blurred)
        return preview, probs, pred

    def reprocess_preview(file_obj, consent: bool, anonymize: bool):
        # Only re-render preview when toggling anonymize without re-uploading
        if not consent or file_obj is None:
            return None
        try:
            pil = Image.open(file_obj.name).convert("RGB")
        except Exception:
            return None
        return _maybe_blur_preview(pil, anonymize)

    def clear_all():
        return (
            None,   # file
            None,   # preview
            {},     # probs
            "",     # label
        )

    with gr.Blocks(title="Human Face Emotion Recognition (HFE)") as demo:
        gr.Markdown("# Human Face Emotion Recognition (HFE)")
        gr.Markdown("Upload an image to classify emotion (consent required).")

        with gr.Accordion("Privacy & Consent (read before proceeding)", open=True):
            gr.Markdown(PRIVACY_TEXT)

        consent = gr.Checkbox(
            label="I have read the Privacy Notice and I consent to process the uploaded image in this session.",
            value=False,
        )

        anonymize = gr.Checkbox(
            label="Blur preview (anonymize UI only)",
            value=True,
        )

        file_in = gr.File(
            label="Upload a face image",
            file_count="single",
            file_types=["image"],
            visible=False,  # gated by consent
        )

        preview = gr.Image(label="Preview (blurred if enabled)", interactive=False)
        probs_out = gr.Label(num_top_classes=5, label="Probabilities")
        pred_out = gr.Textbox(label="Predicted Emotion", interactive=False)
        clear_btn = gr.Button("Clear & Delete from Session")

        consent.change(
            fn=toggle_uploader,
            inputs=consent,
            outputs=[file_in, preview, probs_out, pred_out],
        )

        file_in.change(
            fn=process_file,
            inputs=[file_in, consent, anonymize],
            outputs=[preview, probs_out, pred_out],
        )

        anonymize.change(
            fn=reprocess_preview,
            inputs=[file_in, consent, anonymize],
            outputs=[preview],
        )

        clear_btn.click(
            fn=clear_all,
            inputs=None,
            outputs=[file_in, preview, probs_out, pred_out],
        )

    demo.launch(debug=False, server_name="0.0.0.0")


# --------------------------- Entry Point ---------------------------
if __name__ == "__main__":
    # 1) Path to your saved model (.keras)
    MODEL_PATH = "/content/hfe_emotion_cnn (2).keras"  # change if needed

    # 2) Must match your training label order EXACTLY
    # Example based on your dataset folders (watch spelling: "Suprise" vs "Surprise")
    CLASS_NAMES = ["Angry", "Fear", "Happy", "Sad", "Suprise"]

    # 3) Must match your training input size
    IMG_SIZE = 128

    # Load model
    model = keras.models.load_model(MODEL_PATH)

    # Launch UI
    launch_gradio(model, CLASS_NAMES, IMG_SIZE)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://09f3e23f1337650dfb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
